In [4]:
!pip install accelerate bitsandbytes datasets einops tfds-nightly tensorflow

     |████████████████████████████████| 92.2 MB 389 kB/s  eta 0:00:01


In [2]:
from jsonformer import Jsonformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b-instruct",  trust_remote_code=True,load_in_8bit=True,
        device_map='auto',
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
)


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda111.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 111
CUDA SETUP: Loading binary /home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda111.so...


/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixP

In [64]:

X = pd.read_csv("created_data_wikilingua.csv")

In [65]:
import re
import ast

import re

def decode_byte_string(input_string):
    # find the byte string using regex
    byte_string_match = re.search(r"b'([^']*)'|b\"([^\"]*)\"", input_string)
    
    if byte_string_match is None:
        return input_string

    byte_string = byte_string_match.group(0)
    
    # replace escaped quotes with unescaped ones
    byte_string = byte_string.replace('\\\'', '\'')
    byte_string = byte_string.replace('\\\"', '\"')
    
    # safely parse the byte string as a Python literal and decode it
    try:
        decoded = ast.literal_eval(byte_string).decode('utf-8')
    except (ValueError, SyntaxError):
        return input_string

    # replace the byte string in the input string with the decoded string
    output_string = input_string.replace(byte_string_match.group(0), decoded)

    return output_string




In [118]:
import numpy as np
from sklearn.metrics import mean_squared_error

class CompareHumantoModel:
    def __init__(self, model, tokenizer, data_for_model):
        self.model = model
        self.tokenizer = tokenizer
        self.data_for_model = data_for_model
        self.json_schema = {
            "type": "object",
            "properties":{
                "question1":{"type": "string"},
                "question2":{"type": "string"},
                "question3":{"type": "string"},
                "question4":{"type": "string"},
                "question5":{"type": "string"},
                "question6":{"type": "string"},
            }
        }

    def generate_answers(self):
        results = []

        for idx, data in self.data_for_model.iterrows():
            summary = data['summary']
            article = data['source']
            prompt = self._create_prompt(article, summary)
            jsonformer = Jsonformer(self.model, self.tokenizer, self.json_schema, prompt)
            generated_data = jsonformer()
            results.append(generated_data)

        return pd.DataFrame(results)

    @staticmethod
    def _create_prompt(article, summary):
        return f"""article:{article}
        summary:{summary}

        return answer for questions- 

        Q1 comprehensible: The summary can be read
        and understood by the rater. (If “No,” the rest
        of the questions will be skipped.)
        Q2 repetition: The summary is free of unnecessarily repeated information.
        Q3 grammar: The summary is grammatically
        correct.
        Q4 attribution: All of the information provided
        by the summary is fully attributable to the
        source article, as defined in (Rashkin et al.,
        2021).
        Q5 main ideas: The summary captures the main
        idea(s) of the source article.
        Q6 conciseness: The summary concisely represents the information in the source article.

        All answers are only Yes/No in the following schema:"""

class MeanSquaredErrorCalculator:
    @staticmethod
    def calculate_mse(df_generated, df_human):
        df_generated.replace({'Yes': 1, 'No': 0}, inplace=True)
        df_human.replace({'Yes': 1, 'No': 0,'Unsure':0}, inplace=True)
        df_human = df_human.reset_index().drop('index',axis=1)

        mse_row_wise = []
        for idx in range(0, len(df_generated)):
            mse_row_wise.append(mean_squared_error(df_generated.iloc[idx].values.flatten(),df_human.iloc[idx].values.flatten()))
        
        return np.mean(mse_row_wise)


Mean Squared Error: 0.2333333333333333


/tmp/ipykernel_1595/784421871.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_human.replace({'Yes': 1, 'No': 0,'Unsure':0}, inplace=True)


In [93]:

X['source'] = X['source'].apply(lambda x:decode_byte_string(x))
X[X.model=='t5_xxl']
data_for_model = X[X.model=='mt5_xxl'][:10]

question_answer_generator = CompareHumantoModel(model, tokenizer, data_for_model)
df_generated = question_answer_generator.generate_answers()

df_human_score = data_for_model[['question1', 'question2', 'question3', 'question4', 'question5','question6']]
mse_calculator = MeanSquaredErrorCalculator()
mse = mse_calculator.calculate_mse(df_generated, df_human_score)

print(f"Mean Squared Error: {mse}")